In [4]:
import einops
from tqdm.notebook import tqdm

from torchsummary import summary

import torch
from torch import nn
import torchvision
import torch.optim as optim
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, RandomHorizontalFlip, RandomCrop

In [5]:
# To show the tqdm progress bar
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [6]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

# Parameters from the paper
patch_size = 16
latent_size = 768
n_channels = 3
num_heads = 12
dropout = 0.1
num_classes = 10
size = 224

epochs = 10
base_lr = 10e-3
weight_decay = 0.03
batch_size = 4

cuda:0


## Implementation of linear projection.

In [7]:
class InputEmbedding(nn.Module):
    def __init__(self, patch_size=patch_size, n_channels=n_channels, device=device, latent_size=latent_size, batch_size=batch_size):
        super(InputEmbedding, self).__init__()
        self.patch_size = patch_size
        self.n_channels = n_channels
        self.device = device
        self.latent_size = latent_size
        self.batch_size = batch_size
        self.input_size = self.patch_size * self.patch_size * self.n_channels

        # Linear projection
        self.linearProjection = nn.Linear(self.input_size, self.latent_size)

        self.class_token = nn.Parameter(torch.randn(self.batch_size, 1, self.latent_size)).to(self.device)

        # Positional embedding
        self.pos_embedding = nn.Parameter(torch.randn(self.batch_size, 1, self.latent_size)).to(self.device)

    def forward(self, input_data):
        input_data = input_data.to(self.device)

        # Patchify the input image
        patches = einops.rearrange(
            input_data, 'b c (h h1) (w w1) -> b (h w) (h1 w1 c)', h1=self.patch_size, w1=self.patch_size)
        
        print(input_data.size())
        print(patches.size())

In [8]:
test_input = torch.randn(1, 3, 224, 224)
test_class = InputEmbedding().to(device)
test_class(test_input)

torch.Size([1, 3, 224, 224])
torch.Size([1, 196, 768])
